In a constraint satisfaction problem we're looking at a

- Static
- Determenistic
- Fully observable
- Discrete
- Single agent

problem.

Given a set of variables with corresponding domains, we have a set of constraints that the variables must satisfy. The solution is then an assigmnement of the variables such that all constraints are fullfilled.

# Constraint Networks

A constraint network is defined by

- A finite set of variables
- A finite set of domains for each variable
- A set of constraints

The objective is to find a solution for the constraint network that complies with all the constraints.

### Definition

A (binary) constraint network is a 3-tuple $\mathcal{C} = \langle V, dom, (R_{uv}) \rangle$ such that

- $V$ is a non-empty and finite set of variables
- $dom$ is a function that assigns a non-empty and finite domain to each variable $v \in V$ and
- $(R_{uv})_{u,v \in V, u \neq v}$ is a family of binary relations (constraints) over $V$ where for all $u \neq v$: $R_{uv} \subseteq dom(u) \times dom(v)$

if $R_{uv} = dom(u) \times dom(v)$ then the constraint is trivial, there are no restrictions.

Constraints networks allow for compact encodings of large sets of assignements. Consider a network with $n$ variables with domains of size $k$, which gives $k^n$ assignments.
For the description as a constraint network, at most $ \begin{pmatrix} n \\ 2 \end{pmatrix} $, i.e., $O(n^2)$ constraints have to be provided. Each constraint in turn consists of at most $O(k^2)$ pairs, which gives an encoding size of $O(n^2 k^2)$. The number of assignements is exponentially larger than the description of the constraint network. As a consequence, such descriptions can be used as inputs of general constraint solvers.

Let $\mathcal{C} = \langle V, dom, (R_{uv}) \rangle$ be a constraint network. A partial assignment of $\mathcal{C}$ (or of $V$) is a function

$$
\alpha: V' \rightarrow \bigcup_{v \in V} dom(v)
$$

with $V' \subseteq V$ and $\alpha(v) \in dom(v)$ for all $v \in V'$.
If $V' = V$, then $\alpha$ is called total assignment.

A partial assignment of $\alpha$ of a constraint network $\mathcal{C}$ is called inconsistent if there are variables $u, v$ such that $\alpha$ is defined for both $u$ and $v$ and $\langle \alpha(u), \alpha(v) \rangle \notin R_{uv}$

In this case, we say $\alpha$ violates the constraint $R_{uv}$. A partial assignment is called consistent if it is not inconsistent.

Let $\mathcal{C}$ be a constraint network. A consistent and total assignment of $\mathcal{C}$ is called a solution of $\mathcal{C}$. If a solution of $\mathcal{C}$ exists, $\mathcal{C}$ is called solvable. If no solution exists, $\mathcal{C}$ is called inconsistent.

Consistent partial assignments $\alpha$ cannot necessarily be extended to a solution. It only means that so far (only the variables where $\alpha$ is defined) no constraint is violated.

It is a NP-complete problem to decide whether a given constraint network is solvable.

Let $\mathcal{C} = \langle V, dom, R_{uv} \rangle$ and $\mathcal{C}' = \langle V, dom', R'_{uv} \rangle$ be constraint networks with equal variable sets $V$. $\mathcal{C}$ is called tighter than $\mathcal{C}'$, in symbols $\mathcal{C} \sqsubseteq \mathcal{C}'$, if

- $dom(v) \subseteq dom'(v)$ for all $v \in V$
- $R_{uv} \subseteq R'_{uv}$ for all $u, v \in V$

If at least one of these subset equations is strict, then $\mathcal{C}$ is called strictly tighter than $\mathcal{C}'$, in symbols $\mathcal{C} \sqsubset \mathcal{C}'$.

Let $\mathcal{C}$ and $\mathcal{C}$ be constraint networks with equal variable sets. $\mathcal{C}$ and $\mathcal{C}'$ are called equivalent, in symbols $\mathcal{C} \equiv \mathcal{C}$, if they have the same solution.

## Naive Backtracking

Without inference, the naive backtracking algorithm is given as

In [ ]:
def NaiveBacktracking(C, alpha):
    V, dom, R_uv := C
    if alpha is inconsistent with C:
        return inconsistent
    if alpha is total assignment:
        return alpha

    select some variable v for which alpha is not defined
    for each d in dom(v) in some order:
        alpha' = alpha U {v -> d}
        alpha'' = NaiveBacktracking(C, alpha')
        if alpha'' not inconsistent:
            return alpha''
    return inconsistent

Backtracking corresponds to DFS with the following state space

- States: Partial assignments
- Initial state: Empty assignment
- Goal states: Consistent total assignment
- Actions: assign$_{v,d}$ assigns value $d \in dom(v)$ to variable $v$
- Action costs: All 0
- Transitions: For each non-total consistent assignment $\alpha$, choose variable $v = select(\alpha)$ that is unassigned in $\alpha$. Transition $\alpha \rightarrow \alpha \cup \{v \mapsto d\}$ for each $d \in dom(v)$

Through the DFS, the state space is a directed tree without duplicates.
Naive Backtracking often has to exhaustively explore similar paths. More Critical variables are not recognized and hence considered for the assignment too late. Decisions that lead to constraint violates are only recognized when all variables involved in the constraint have been assigned.

Backtracking does not specify the order in which the variables are considered for the assignment. It also does not specify in which order the values of the selected variable $v$ are considered.

We distinguish
- Static orders: Fixed prior to search
- Dynamic orders: Selected variable or value order depends on the search state

Two common variable ordering criteria
- Minimum remaining values: Prefer variables that have small domains.
- Most constraining variable: Prefer variables contained in many nontrivial constraints.

Let $\mathcal{C} = \langle V, dom, (R_{uv}) \rangle$ be a constraint network. For variables $v \neq v'$ and values $d \in dom(v), d' \in dom(v')$, the assignment $v \mapsto d$ is in conflict with $v' \mapsto d'$ if $\langle d, d' \rangle \notin R_{vv'}$

The value ordering criterion for a partial assignment $\alpha$ and selected variable $v$, is that we prefer values $d \in dom(v)$ such that $v \mapsto d$ causes as few conflicts as possible with variables that are unassigned in $\alpha$.

## Inference

Here we derive additional constraints that are implied by the given constraints, i.e., that are satisfied in all solutions.

For a given constraint network $\mathcal{C}$, replace $\mathcal{C}$ with an equivalent, but tighter constraint network. The more complex the inference and the more often inference is applied, the smaller the resulting state space but the higher the complexity per seach node.

We can apply inference once, as a preprocessing before search or combine it with search, where before each recursive call, due to the assignments of variables $v \mapsto d$, more inference is possible and during backtrackng, where derived constraint have to be retracted.

In [ ]:
def NaiveBacktrackingWithInference(C, alpha):
    V, dom, R_uv := C
    if alpha is inconsistent with C:
        return inconsistent
    if alpha is total assignment:
        return alpha
    C' := <V, dom', R'> := copy of C
    apply inference to C'
    if dom'(v) =/= for all variables v:
    select some variable v for which alpha is not defined
        for each d in dom(v) in some order:
            alpha' = alpha U {v -> d}
            dom'(v) = {d}
            alpha'' = NaiveBacktrackingWithInference(C', alpha')
            if alpha'' not inconsistent:
                return alpha''
    return inconsistent

## Forward Checking

Let $\alpha$ be a partial assignment. Inference: For all unassigned variables $v$ in $\alpha$, remove all values from the domain of $v$ that are in conflict with already assigned variable/value pairs in $\alpha$.

Properties:
- Correct inference method (retains equivalence)
- Affects domains but not constraints
- Consistency check at the beginning of the backtracking procedure no longer needed
- Cheap but useful

## Arc Consistency

Let $\mathcal{C} = \langle V, dom, (R_{uv}) \rangle$ be a constraint network.
1. The variable $v \in V$ is arc consistent with respect to another variable $v' \in V$, if for every value $d \in dom(v)$ there exist a value $d'\in dom(v')$ with $\langle d,d' \rangle \in R_{vv'}$.
2. The constraint network $\mathcal{C}$ is arc consistent if every variable $v \in V$ is arc consistent with respect to every other variable $v' \in V$.

This can be enforced by removing values from the $dom(v)$ that violate the arc consistency of $v$ w.r.t $v'$. This method is more powerful than forward checking.

In [ ]:
def revise(C, v, v'):
    <V, dom, R_uv> = C
    for each d in dom(v):
        if there is no d' in dom(v') with (d,d') in R_vv':
            remove d from dom(v)

# Enforcing arc consistency
def AC1(C):
    <V, dom, R_uv> = C
    repeat:
        for each nontrivial constraint R_uv:
            revise(C, u, v)
            revise(C, v, u)
    until no domain has changed in this iteration

The problem is that AC is rather inefficient. It has a running time of $O(n \cdot e \cdot k^3)$, with $n$ variables, $e$ nontrivial constraints and maximal domain size $k$. Variable pairs are often checked again and again altough their domains have remained unchanged.

In [ ]:
def AC3(C):
    <V, dom, R_uv> = C
    queue = []
    for each nontrivial constraint R_uv:
        insert u, v into queue
        insert v, u into queue
    while queue != []:
        remove arbitrary element (u, v) from queue
        revise(C, u, v)
        if dom(u) changed in the call to revise
            for each w in V \ {u, v} where R_wu is nontrivial:
                insert w, u into queue

The time complexity of AC-3 is $O(e \cdot k^3)$

### Proof

Consider a pair $\langle u, v \rangle$ such that there exists a nontrivial constraint $R_{uv}$ or $R_{vu}$. (There are at most 2e of such pairs.)
Every time this pair is inserted to the queue (except for the first time) the domain of the second variable has just been reduced.
This can happen at most $k$ times.
Hence every pair $\langle u, v \rangle$ is inserted into the queue at most $k + 1$ times $\rightarrow$ at most $O(e \cdot k)$ insert operations in total.
This bounds the number of while iterations by $O(e \cdot k)$, giving an overall time complexity of $O(e\cdot k) \cdot O(k^2) = O(e\cdot k^3)$.

Futher consistencies are

- Path Consistency

For every joint assignment to variables $u$ and $v$, there must be a suitable assignment to every third variable $w$. If not, then remove pairs of values of $u$ and $v$ for which no suitable "partner" assignment to $w$ exists.

- $i$ - Consistency

For every joint assignment to variables $v_1, ..., v_{i-1}$, there must be a suitable assignment to every $i$-th variable $v_i$. If not, then remove value tuples of $v_1, ..., v_{i-1}$ for which no suitable "partner" assignment for $v_i$ exists.

## Path Consistency

Let $\mathcal{C} = \langle V, dom, (R_{uv}) \rangle$ be a constraint network.
1. Two different variables $u, v \in V$ are path consistent w.r.t a third variable $w \in V$ if for all values $d_u \in dom(u)$, $d_v \in dom(v)$ with $\langle d_u, d_v \rangle \in R_{uv}$ there is a value $d_w \in dom(w)$ with $\langle d_u, d_w \rangle \in R_{uw}$ and $\langle d_v, d_w \rangle \in R_{vw}$.
2. The constraint network $\mathcal{C}$ is path consistent if for all triples of different variables $u,v,w$ the variables $u$ and $v$ are path consistent w.r.t $w$.

In [ ]:
def revise_3(C, u, v, w):
    V, dom, R_uv = C
    for each <d_u, d_v> in R_uv:
        if d_w there is no dom(w) with <d_u, d_w> in R_uw and <d_v, d_w> in R_vw:
            del <d_u, d_v> from R_uv

In [ ]:
def PC2(C):
    <V, dom, R_uv> = C
    queue = []
    for each set of two variables {u,v}:
        for each w in V \ {u, v}:
            insert <u, v, w> in queue
    while queue != []:
        remove arbitrary element (u, v, w) from queue
        revise_3(C, u, v, w)
        if R_uv changed in the call to revise_3
            for each w' in V \{u, v}:
                insert <w', u, v> in queue
                insert <w', v, u> in queue

PC2 enforces path consistency. It has a time complexity of $O(n^3k^5)$

## Constraint Graphs

To solve a constraint network consisting of $n$ variables and $k$ values, $k^n$ assignments must be considered. Inference can help exploring this space but not always. Many practical relevant constraint networks are efficiently solvable if their structure is taken into account.

Let $\mathcal{C} = \langle V, dom, (R_{uv})) \rangle$ be a constraint network. The constraint graph of $\mathcal{C}$ is the graph whose vertices are $V$ and which contains an edge $\{u, v\}$ iff $R_{uv}$ is a nontrivial constraint.

### Unconnected Graphs

- Proposition
If the constraint graph of $\mathcal{C}$ has multiple connected components, the subproblem induced by each component can be solved seperately. The union of the solutions of these subproblems is a solution for $\mathcal{C}$.
- Proof
A total assignment consisting of combined subsolutions satisfies all constraints that occur within the subproblems. All constraints between two subproblems are trivial.

## Trees

Let $\mathcal{C}$ be a constraint network with $n$ variables and maximal domain size $k$ whose constraint graph is a tree or forest (no cycles). Then we can solve $\mathcal{C}$ or prove that no solution exists in time $O(n k^2)$.

### Algorithm

- Build a directed tree for the constraint graph. Select an arbitrary variable as the root.
- Order variables $v_1, ..., v_n$ such that parents are ordered before their children.
- For $i \in \langle n, n-1, ..., 2 \rangle$ call $revise(v_{parent(i)}, v_i)$. Each variable is arc consistent w.r.t its children
- If a domain becomes empty, the problem is unsolvable.
- Otherwise, solve with BacktrackingWithInference, variable order $v_1,...,v_n$ and forward checking. Solution is found without backtracking steps.

## Decomposition Methods

What if the constraint graph is not a tree and does not decompose into several components

1. Conditioning
2. Tree decomposition

## Conditioning

Apply backtracking with forward checking until the constraint graph restricted to the remaining unassigned variables decomposes or is a tree.

- Cutset conditioning: Choose variable order such that early variables form a small cutset (set of variables such that removing these variables results in an acyclic constraint graph).
- Time complexity: $n$ variables, $m < n$ in cutset, maximal domain size $k$. $O(k^m \cdot (n - m)k^2)$ (Finding the optimal custets is a NP problem)

## Tree Decomposition

- Decompose constraint network into smaller subproblems
- Find best solutions for subproblem
- Build overall solution based on the subsolutions

Choose subproblems in a way that the constraint graph of the meta constraint network (solution network based on subsolutions) is a tree / forest. Build the overall solution with efficient tree algorithm.

Consider a constraint network $\mathcal{C}$ with variables $V$. A tree decomposition of $\mathcal{C}$ is a graph $\mathcal{T}$ with the following properties:

- Every vertex of $\mathcal{T}$ corresponds to a subset of the variables $V$. Such a vertex (and corresponding variable set) is called a subproblem of $\mathcal{C}$.
- Every variable of $V$ appears in at least one subproblem of $\mathcal{T}$.
- For every nontrivial constraint $R_{uv}$ of $\mathcal{C}$, the variables $u$ and $v$ appear together in at least one subproblem in $\mathcal{T}$.

- For each variable $v \in V$, let $\mathcal{T}_v$ be the set of vertices corresponding to the subproblems that contain $v$.
- For each variable $v$, the set $\mathcal{T}_v$ is connected, i.e. each vertex in $\mathcal{T}_v$ is reachable from every other vertex in $\mathcal{T}_v$ without visiting vertices not contained in $\mathcal{T}_v$.
- $\mathcal{T}$ is acyclic

## Meta constraint network

meta constraint network $\mathcal{C}^{\mathcal{T}} = \langle V^{\mathcal{T}}, dom^{\mathcal{T}}, (R_{uv}^{\mathcal{T}}) \rangle$ based on tree decomposition $\mathcal{T}$
- $V^{\mathcal{T}}$ = Vertices of $\mathcal{T}$ (subproblems of $\mathcal{C}$ occuring in $\mathcal{T}$)
- $dom^{\mathcal{T}}(v)$ = set of solutions of subproblem $v$
- $R_{uv}^{\mathcal{T}} = \{\langle s, t \rangle | s,t$ compatible solutions of subproblems $ u, v \}$

Solutions of two subproblems are called compatible if all overlapping variables are assigned identically.

## Algorithm

- Find all solutions for all subproblems in the decomposition and build a tree-like meta constraint network.
- Constraint in meta constraint network: Subsolutions must be compatible
- Solve meta constraint network with an algorithm for tree-like networks.

The goal is that each subproblem has as few variables as possible.
- Crucial: subproblem $V'$ in $\mathcal{T}$ with highest number of variables
- Number of variables in $V'$ - 1 is called the width of the decomposition
- Best width over all decompositions is the tree width of the constraint graph. (NP complete)

The time complexity of solving algorithm based on the tree decomposition is $O(nk^{w+1})$, where $w$ is the width of the decomposition